# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [7]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
city_data_csv = 'output_data/cities.csv'
city_data = pd.read_csv(city_data_csv)
city_data_df = pd.DataFrame(city_data)
city_data_df


,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed,Country
0,poum,70.41,-20.2333,164.0167,71,99,9.78,NC
1,alofi,78.69,-19.0595,-169.9187,61,28,11.50,NU
2,kapaa,86.32,22.0752,-159.3190,72,20,5.01,US
3,hilo,82.13,19.7297,-155.0900,75,40,11.50,US
4,ushuaia,31.66,-54.8000,-68.3000,100,90,33.38,AR
...,...,...,...,...,...,...,...,...
532,charters towers,73.85,-20.1000,146.2667,31,0,9.98,AU
533,amapa,73.54,1.0000,-52.0000,87,56,3.83,BR
534,verkhnevilyuysk,70.95,63.4453,120.3167,54,100,10.22,RU
535,udalguri,82.18,26.7667,92.1333,87,93,6.24,IN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)


In [10]:
locations = city_data_df[['Latitude', 'Longitude']]
humidity = city_data_df['Humidity']
figure = gmaps.figure()


In [11]:
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating = False,
                              max_intensity = 250, point_radius = 4)

figure.add_layer(heatmap)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#narrow max temp > 70 and < 80, wind speed less than 10, and zero cloudiness
narrowed_cities = city_data_df.loc[(city_data_df['Max Temperature'] < 80) & (city_data_df["Max Temperature"]                             
                                                                             > 70) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloud %'] == 0)]
print(len(narrowed_cities))
narrowed_cities


13


,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed,Country
169,olbia,77.05,40.9214,9.4856,50,0,4.61,IT
177,sabzevar,79.48,36.2126,57.6819,21,0,4.59,IR
206,khash,76.91,28.2211,61.2158,16,0,2.98,IR
257,changji,79.59,44.0167,87.3167,36,0,0.00,CN
332,podolsk,70.02,55.4242,37.5547,64,0,4.47,RU
350,bundaberg,70.09,-24.8500,152.3500,35,0,8.05,AU
354,methoni,79.21,36.8197,21.7049,63,0,3.33,GR
388,mizdah,77.65,31.4451,12.9801,53,0,5.70,LY
394,urubicha,71.87,-15.3833,-62.9500,48,0,1.90,BO
401,dubrovnik,73.04,42.6481,18.0922,60,0,4.61,HR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:

vacation_locations = narrowed_cities[['City','Latitude', 'Longitude','Country']]
vacation_lat = narrowed_cities['Latitude']
vacation_lon = narrowed_cities["Longitude"]
vacation_country = narrowed_cities['Country']
hotel_df = pd.DataFrame(narrowed_cities)
hotel_df["Hotel Name"] = ''
hotel_df
#this section is not working, I will try another approach. 
#Perhaps it is not working because of vacation_lat/lon not being iterated through.

#hotel_search_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={vacation_lat},{vacation_lon}&radius=5000&types=lodging&key={g_key}'
#for city in hotel_df.iterrows():
   # response = requests.get(hotel_search_url).json()
   # print(response)

,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed,Country,Hotel Name
169,olbia,77.05,40.9214,9.4856,50,0,4.61,IT,
177,sabzevar,79.48,36.2126,57.6819,21,0,4.59,IR,
206,khash,76.91,28.2211,61.2158,16,0,2.98,IR,
257,changji,79.59,44.0167,87.3167,36,0,0.00,CN,
332,podolsk,70.02,55.4242,37.5547,64,0,4.47,RU,
350,bundaberg,70.09,-24.8500,152.3500,35,0,8.05,AU,
354,methoni,79.21,36.8197,21.7049,63,0,3.33,GR,
388,mizdah,77.65,31.4451,12.9801,53,0,5.70,LY,
394,urubicha,71.87,-15.3833,-62.9500,48,0,1.90,BO,
401,dubrovnik,73.04,42.6481,18.0922,60,0,4.61,HR,


In [24]:
#try using params to get the nearest hotels
params = {
    'radius' : 5000,
    'types'  : 'lodging',
    'key'    : g_key
}

places_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

for index, place in hotel_df.iterrows():
    lat = place['Latitude']
    lon = place['Longitude']
    
    params['location'] = f'{lat},{lon}'
    try:
        hotel = requests.get(places_url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = hotel['results'][0]['name']
    except:
        print('No result')

    

No result


In [25]:
hotel_df

,City,Max Temperature,Latitude,Longitude,Humidity,Cloud %,Wind Speed,Country,Hotel Name
169,olbia,77.05,40.9214,9.4856,50,0,4.61,IT,Hotel Centrale Olbia
177,sabzevar,79.48,36.2126,57.6819,21,0,4.59,IR,پزشکی قانونی سبزوار
206,khash,76.91,28.2211,61.2158,16,0,2.98,IR,دفتر پیشخوان دولت جمال زهی (خالد)
257,changji,79.59,44.0167,87.3167,36,0,0.00,CN,Lvye Holiday Resort
332,podolsk,70.02,55.4242,37.5547,64,0,4.47,RU,Hotel Pahra
350,bundaberg,70.09,-24.8500,152.3500,35,0,8.05,AU,Takalvan Motel
354,methoni,79.21,36.8197,21.7049,63,0,3.33,GR,Methoni Beach Hotel
388,mizdah,77.65,31.4451,12.9801,53,0,5.70,LY,محمدالمبروك الاريقط
394,urubicha,71.87,-15.3833,-62.9500,48,0,1.90,BO,
401,dubrovnik,73.04,42.6481,18.0922,60,0,4.61,HR,Hotel Dubrovnik Palace


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

figure.add_layer(layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))